# David's Section 2 - End to end Machine Learning Project

*This is my work through version of chapter 2*

**Download Data**

The first thing we need to do is download the data and it's best we do this in a function so that if the data updates then we can easily run a command to donwload it again rather than use a web browser manually. We will also define where the data is saved meaning that it is always stored in the same place!

In [3]:
import os
import tarfile
from six.moves import urllib

# Setting constants, hence capitalised - Remember all these direcotries are based on current directory!!

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = "datasets/housing"
HOUSING_URL = DOWNLOAD_ROOT + HOUSING_PATH + "/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
#   If housing path does not exist then use os library to create folder structure
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
#   Defining where the tgz is stored
    tgz_path = os.path.join(housing_path, "housing.tgz")
#   We have set it up so that the path we save to is similar to the url to get the files from
    urllib.request.urlretrieve(housing_url, tgz_path)
#   create tarfile object to open, store and save file.
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

When you call the **fetch_housing_data()** function it creates datasets/housing directory in your workspace, downoads the housing.tgz file and extracts the the housing.csv file from this directory!!

In [2]:
# We need to run our function
fetch_housing_data()



# Loading Data and getting visbility on data
Let's use Pandas to load the data!

In [6]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [15]:
# save dataframe as output of function and read top 5 rows of data
housing = load_housing_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


The info function is good to get information suahc as the number of rows in the data. **Notice that the total_bedrooms has only 20,433 nonnull values, meaning that 207 districts are missing this feature**.

In [16]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
longitude             20640 non-null float64
latitude              20640 non-null float64
housing_median_age    20640 non-null float64
total_rooms           20640 non-null float64
total_bedrooms        20433 non-null float64
population            20640 non-null float64
households            20640 non-null float64
median_income         20640 non-null float64
median_house_value    20640 non-null float64
ocean_proximity       20640 non-null object
dtypes: float64(9), object(1)
memory usage: 1.6+ MB
